Lambda School Data Science, Unit 2: Predictive Modeling

# Kaggle Challenge, Module 1

## Assignment
- [X] Do train/validate/test split with the Tanzania Waterpumps data.
- [X] Define a function to wrangle train, validate, and test sets in the same way. Clean outliers and engineer features. (For example, [what other columns have zeros and shouldn't?](https://github.com/Quartz/bad-data-guide#zeros-replace-missing-values) What other columns are duplicates, or nearly duplicates? Can you extract the year from date_recorded? Can you engineer new features, such as the number of years from waterpump construction to waterpump inspection?)
- [ ] Select features. Use a scikit-learn pipeline to encode categoricals, impute missing values, and fit a decision tree classifier.
- [ ] Get your validation accuracy score.
- [ ] Get and plot your feature importances.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.


## Stretch Goals

### Reading

- A Visual Introduction to Machine Learning
  - [Part 1: A Decision Tree](http://www.r2d3.us/visual-intro-to-machine-learning-part-1/)
  - [Part 2: Bias and Variance](http://www.r2d3.us/visual-intro-to-machine-learning-part-2/)
- [Decision Trees: Advantages & Disadvantages](https://christophm.github.io/interpretable-ml-book/tree.html#advantages-2)
- [How a Russian mathematician constructed a decision tree — by hand — to solve a medical problem](http://fastml.com/how-a-russian-mathematician-constructed-a-decision-tree-by-hand-to-solve-a-medical-problem/)
- [How decision trees work](https://brohrer.github.io/how_decision_trees_work.html)
- [Let’s Write a Decision Tree Classifier from Scratch](https://www.youtube.com/watch?v=LDRbO9a6XPU) — _Don’t worry about understanding the code, just get introduced to the concepts. This 10 minute video has excellent diagrams and explanations._
- [Random Forests for Complete Beginners: The definitive guide to Random Forests and Decision Trees](https://victorzhou.com/blog/intro-to-random-forests/)


### Doing
- [ ] Add your own stretch goal(s) !
- [ ] Try other [scikit-learn imputers](https://scikit-learn.org/stable/modules/impute.html).
- [ ] Try other [scikit-learn scalers](https://scikit-learn.org/stable/modules/preprocessing.html).
- [ ] Make exploratory visualizations and share on Slack.


#### Exploratory visualizations

Visualize the relationships between feature(s) and target. I recommend you do this with your training set, after splitting your data. 

For this problem, you may want to create a new column to represent the target as a number, 0 or 1. For example:

```python
train['functional'] = (train['status_group']=='functional').astype(int)
```



You can try [Seaborn "Categorical estimate" plots](https://seaborn.pydata.org/tutorial/categorical.html) for features with reasonably few unique values. (With too many unique values, the plot is unreadable.)

- Categorical features. (If there are too many unique values, you can replace less frequent values with "OTHER.")
- Numeric features. (If there are too many unique values, you can [bin with pandas cut / qcut functions](https://pandas.pydata.org/pandas-docs/stable/getting_started/basics.html?highlight=qcut#discretization-and-quantiling).)

You can try [Seaborn linear model plots](https://seaborn.pydata.org/tutorial/regression.html) with numeric features. For this problem, you may want to use the parameter `logistic=True`

You do _not_ need to use Seaborn, but it's nice because it includes confidence intervals to visualize uncertainty.

#### High-cardinality categoricals

This code from a previous assignment demonstrates how to replace less frequent values with 'OTHER'

```python
# Reduce cardinality for NEIGHBORHOOD feature ...

# Get a list of the top 10 neighborhoods
top10 = train['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10,
# replace the neighborhood with 'OTHER'
train.loc[~train['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'
test.loc[~test['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'
```



In [1]:
# If you're in Colab...
import os, sys
in_colab = 'google.colab' in sys.modules

if in_colab:
    # Install required python packages:
    # category_encoders, version >= 2.0
    # pandas-profiling, version >= 2.0
    # plotly, version >= 4.0
    !pip install --upgrade category_encoders pandas-profiling plotly
    
    # Pull files from Github repo
    os.chdir('/content')
    !git init .
    !git remote add origin https://github.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge.git
    !git pull origin master
    
    # Change into directory for module
    os.chdir('module1')

     |████████████████████████████████| 92kB 5.8MB/s 
     |████████████████████████████████| 133kB 15.8MB/s 
     |████████████████████████████████| 7.1MB 46.5MB/s 
     |████████████████████████████████| 614kB 42.9MB/s 
     |████████████████████████████████| 225kB 56.6MB/s 
     |████████████████████████████████| 768kB 43.8MB/s 
     |████████████████████████████████| 194kB 60.1MB/s 
     |████████████████████████████████| 51kB 26.0MB/s 
     |████████████████████████████████| 737kB 26.4MB/s 
     |████████████████████████████████| 51kB 26.3MB/s 
  Created wheel for pandas-profiling: filename=pandas_profiling-2.3.0-py2.py3-none-any.whl size=145035 sha256=ecc59ca7ca646ae80a56fc5b1d578fef3f0752b546ca1a03307aa8f6e8a76251
  Stored in directory: /root/.cache/pip/wheels/ce/c7/f1/dbfef4848ebb048cb1d4a22d1ed0c62d8ff2523747235e19fe
  Created wheel for htmlmin: filename=htmlmin-0.1.12-cp36-none-any.whl size=27084 sha256=d852bf3bc48b8d1029f5e5d4d114d7f7047cbabcb66ccf644d75525373b524c8
  Stored

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

train = pd.merge(pd.read_csv('../data/tanzania/train_features.csv'), 
                 pd.read_csv('../data/tanzania/train_labels.csv'))
test = pd.read_csv('../data/tanzania/test_features.csv')
sample_submission = pd.read_csv('../data/tanzania/sample_submission.csv')

train.shape, test.shape

((59400, 41), (14358, 40))

In [3]:
import plotly.express as px
px.scatter(train, x='longitude', y='latitude', color='status_group', opacity=0.1)

In [4]:
train[['longitude', 'latitude']].describe()

longitude      latitude
count  59400.000000  5.940000e+04
mean      34.077427 -5.706033e+00
std        6.567432  2.946019e+00
min        0.000000 -1.164944e+01
25%       33.090347 -8.540621e+00
50%       34.908743 -5.021597e+00
75%       37.178387 -3.326156e+00
max       40.345193 -2.000000e-08

In [0]:
import numpy as np

def wrangle(x):
  X= x.copy()
  X['latitude'] = X['latitude'].replace(-2e-08, 0)
  cols_with_zeros = ['longitude', 'latitude']
  for col in cols_with_zeros:
      X[col] = X[col].replace(0, np.nan)  
  X = X.drop(columns='quantity_group')      
  return(X)
  

In [0]:
train = wrangle(train)

In [17]:
px.scatter(train, x='longitude', y='latitude', color='status_group', opacity=0.1)

In [11]:
train[['longitude', 'latitude']].describe()

longitude      latitude
count  57588.000000  57588.000000
mean      35.149669     -5.885572
std        2.607428      2.809876
min       29.607122    -11.649440
25%       33.285100     -8.643841
50%       35.005943     -5.172704
75%       37.233712     -3.372824
max       40.345193     -0.998464

In [46]:
test = test.drop(columns='id')
test.shape


(14358, 38)

In [0]:
from sklearn.model_selection import train_test_split

train_features = train.drop(columns=['status_group','id']).columns

X_train, X_val, y_train, y_val = train_test_split(train[train_features], train['status_group'], train_size=0.8, test_size=0.2, stratify=train['status_group'])

In [48]:
X_train.shape

(47520, 38)

In [49]:
X_train.head()

amount_tsh date_recorded  ...     waterpoint_type  waterpoint_type_group
58538         0.0    2011-03-28  ...  communal standpipe     communal standpipe
32710       500.0    2011-02-18  ...  communal standpipe     communal standpipe
11757       200.0    2011-04-18  ...  communal standpipe     communal standpipe
20305         0.0    2011-07-10  ...  communal standpipe     communal standpipe
34960         0.0    2011-02-23  ...  communal standpipe     communal standpipe

[5 rows x 38 columns]

In [50]:
type(X_train['date_recorded'])

pandas.core.series.Series

In [0]:
def inspect(li):
  return(int(li.split('-')[0]))


X_train['time_inspect'] = X_train.date_recorded.apply(inspect)
X_val['time_inspect'] = X_val.date_recorded.apply(inspect)
test['time_inspect'] = test.date_recorded.apply(inspect)


In [53]:
test.head()

amount_tsh date_recorded  ... waterpoint_type_group  time_inspect
0         0.0    2013-02-04  ...                 other          2013
1         0.0    2013-02-04  ...    communal standpipe          2013
2         0.0    2013-02-01  ...                 other          2013
3         0.0    2013-01-22  ...                 other          2013
4       500.0    2013-03-27  ...    communal standpipe          2013

[5 rows x 39 columns]

In [0]:
def nans(li):
  if li == 0:
    li = np.nan
  else:
    li = li
  return(li)

X_train['construction_year'] = X_train['construction_year'].apply(nans)
test['construction_year'] = test['construction_year'].apply(nans)
X_val['construction_year'] = X_val['construction_year'].apply(nans)

In [55]:
X_train['time_inspect'] = X_train.time_inspect - X_train.construction_year
X_val['time_inspect'] = X_val.time_inspect - X_val.construction_year
test['time_inspect'] = test.time_inspect - test.construction_year
X_train.head()

amount_tsh date_recorded  ... waterpoint_type_group  time_inspect
58538         0.0    2011-03-28  ...    communal standpipe          41.0
32710       500.0    2011-02-18  ...    communal standpipe           3.0
11757       200.0    2011-04-18  ...    communal standpipe          15.0
20305         0.0    2011-07-10  ...    communal standpipe           NaN
34960         0.0    2011-02-23  ...    communal standpipe          26.0

[5 rows x 39 columns]

In [56]:
X_val.head()

amount_tsh date_recorded  ... waterpoint_type_group  time_inspect
42711       200.0    2011-03-25  ...    communal standpipe           1.0
9554          0.0    2013-02-11  ...             hand pump           3.0
34443         0.0    2011-03-06  ...    communal standpipe          13.0
5926         50.0    2011-02-24  ...    communal standpipe           1.0
11905         0.0    2013-01-30  ...                 other          42.0

[5 rows x 39 columns]

In [59]:
features = X_train.columns
target = 'status_group'

Index(['amount_tsh', 'date_recorded', 'funder', 'gps_height', 'installer',
       'longitude', 'latitude', 'wpt_name', 'num_private', 'basin',
       'subvillage', 'region', 'region_code', 'district_code', 'lga', 'ward',
       'population', 'public_meeting', 'recorded_by', 'scheme_management',
       'scheme_name', 'permit', 'construction_year', 'extraction_type',
       'extraction_type_group', 'extraction_type_class', 'management',
       'management_group', 'payment', 'payment_type', 'water_quality',
       'quality_group', 'quantity', 'source', 'source_type', 'source_class',
       'waterpoint_type', 'waterpoint_type_group', 'time_inspect'],
      dtype='object')

In [0]:
import category_encoders as ce 
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression

pipeline = make_pipeline(
    ce.OneHotEncoder(use_cat_names=True),
    SimpleImputer(),
    LogisticRegression(multi_class='auto', solver='lbfgs', n_jobs=-1)
)

In [0]:
pipeline.fit(X_train, y_train)

print('Validation Accuracy', pipeline.score(X_val, y_val))